In [1]:
import torch
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
from wandb_addons.diffusers import StableDiffusionXLCallback

In [2]:
base_pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)

base_pipeline = base_pipeline.to("cuda")
base_pipeline.enable_xformers_memory_efficient_attention()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

num_inference_steps = 50

callback = StableDiffusionXLCallback(
    pipeline=base_pipeline,
    prompt=prompt,
    wandb_project="diffusers-2",
    wandb_entity="geekyrakshit",
    weave_mode=False,
    num_inference_steps=num_inference_steps,
    initial_stage_name="base",
)

image = base_pipeline(
    prompt=prompt,
    output_type="latent",
    num_inference_steps=num_inference_steps,
    callback=callback
).images[0]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: geekyrakshit. Use `wandb login --relogin` to force relogin


  0%|          | 0/50 [00:00<?, ?it/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [ ]:
refiner_pipeline = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base_pipeline.text_encoder_2,
    vae=base_pipeline.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner_pipeline = refiner_pipeline.to("cuda")
refiner_pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
image = refiner_pipeline(prompt=prompt, image=image[None, :]).images[0]

In [ ]:
image